# Creating Positional Encodings

One of the main advantage of this library is the ability to easily incorporate novel positional encodings on the node, edge and graph level. The positional encodings are computed and feed into respective encoders and then the hidden embeddings from all pe encoders are pooled (according to if they are node, edge, or graph level) and then feed into the GNN layers as features. The designs allow any combination of positional encodings to be used by modifying the configuration file. For more details on the data processing part, please visit the [design page of the doc](https://valence-discovery.github.io/goli/design.html).

Here is the workflow for computing and processing positional encoding in the library:
1. modify the arguments in the configuration file

2. compute the raw positional encoding from the graph in [`goli/features/positional_encoding.py`](https://valence-discovery.github.io/goli/api/goli.features.html#goli.features.positional_encoding) (from the [`graph positional encoder`](https://valence-discovery.github.io/goli/api/goli.features.html#goli.features.positional_encoding.graph_positional_encoder))

3. feed the raw positional encoding into the respective (specialized) encoders in [`goli/nn/encoders`](https://valence-discovery.github.io/goli/api/goli.nn/encoders.html). For example, a simple [`MLP positional encoder`](https://valence-discovery.github.io/goli/api/goli.nn/encoders.html#goli.nn.encoders.mlp_encoder) can be found. 

4. Output the hidden embeddings of pe from the encoders in their respective output keys: `feat`(node feature), `edge_feat`(edge feature), `graph_feat`(graph feature) and potentially other keys if needed such as `nodepair_feat` 

5. pool the hidden embeddings with same keys together: for example, all output with `feat` key will be pooled together

6. Construct the [`PyG Batch`](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.data.Batch.html#torch_geometric.data.Batch), batch of graphs, each contain the output keys seen above, ready for use in the [GNN layers](https://valence-discovery.github.io/goli/api/goli.nn/pyg_layers.html) 

Since this library is built using PyG, we recommend looking at their [Docs](https://pytorch-geometric.readthedocs.io/en/latest/) and [Tutorials](https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html) for more info. 

We start by editing the configuration file first.


### Table of content
1. [edit the config file](#Edit-the-Configuration-File)

## Edit the Configuration File



```
pos_encoding_as_features:
    pos_types:
      la_pos: &pos_enc  #use same name as pe_encoder
        pos_type: laplacian_eigvec_eigval #laplacian_eigvec
        num_pos: 3
        normalization: "none"
        disconnected_comp: True
      rw_pos: #use same name as pe_encoder
        pos_type: rwse
        ksteps: 16
```

In [1]:
%load_ext autoreload
%autoreload 2

import torch
import dgl
from copy import deepcopy

from goli.nn.dgl_layers import BaseDGLLayer
from goli.nn.base_layers import FCLayer
from goli.utils.decorators import classproperty


_ = torch.manual_seed(42)

Using backend: pytorch


## Pre-defining test variables

We define below a small batched graph on which we can test the created layers

In [2]:
in_dim = 5          # Input node-feature dimensions
out_dim = 11        # Desired output node-feature dimensions
in_dim_edges = 13   # Input edge-feature dimensions

# Let's create 2 simple graphs. Here the tensors represent the connectivity between nodes
g1 = dgl.graph((torch.tensor([0, 1, 2]), torch.tensor([1, 2, 3])))
g2 = dgl.graph((torch.tensor([0, 0, 0, 1]), torch.tensor([0, 1, 2, 0])))

# We add some node features to the graphs
g1.ndata["h"] = torch.rand(g1.num_nodes(), in_dim, dtype=float)
g2.ndata["h"] = torch.rand(g2.num_nodes(), in_dim, dtype=float)

# We also add some edge features to the graphs
g1.edata["e"] = torch.rand(g1.num_edges(), in_dim_edges, dtype=float)
g2.edata["e"] = torch.rand(g2.num_edges(), in_dim_edges, dtype=float)

# Finally we batch the graphs in a way compatible with the DGL library
bg = dgl.batch([g1, g2])
bg = dgl.add_self_loop(bg)

# The batched graph will show as a single graph with 7 nodes
print(bg)

Graph(num_nodes=7, num_edges=14,
      ndata_schemes={'h': Scheme(shape=(5,), dtype=torch.float64)}
      edata_schemes={'e': Scheme(shape=(13,), dtype=torch.float64)})


## Creating a simple layer

Here, we will show how to create a GNN layer that does a mean aggregation on the neighbouring features.

First, for the layer to be fully compatible with the flexible architecture provided by `FeedForwardDGL`, it needs to inherit from the class `BaseDGLLayer`. This base-layer has multiple virtual methods that must be implemented in any class that inherits from it.

The virtual methods are below

- `layer_supports_edges`: We want to return `False` since our layer doesn't support edges
- `layer_inputs_edges`: We want to return `False` since our layer doesn't input edges
- `layer_outputs_edges`: We want to return `False` since our layer doesn't output edges
- `layer_outdim_factor`: We want to return `1` since the output dimension does not depend on internal parameters.

The example is given below

In [3]:
class SimpleMeanLayer(BaseDGLLayer):
    def __init__(self, in_dim, out_dim, activation, dropout, normalization):
        # Initialize the parent class
        super().__init__(   in_dim=in_dim, out_dim=out_dim, activation=activation,
                            dropout=dropout, normalization=normalization)

        # Create the layer with learned parameters
        self.layer = FCLayer(in_dim=in_dim, out_dim=out_dim)

    def forward(self, g, h):
        # We first apply the mean aggregation
        g.ndata["h"] = h
        g.update_all(message_func=dgl.function.copy_u("h", "m"), 
                    reduce_func=dgl.function.mean("m", "h"))

        # Then we apply the FCLayer, and the non-linearities
        h = g.ndata["h"]
        h = self.layer(h)
        h = self.apply_norm_activation_dropout(h)
        return h

    # Finally, we define all the virtual properties according to how
    # the class works
    @classproperty
    def layer_supports_edges(cls):
        return False

    @property
    def layer_inputs_edges(self):
        return False

    @property
    def layer_outputs_edges(self):
        return False

    @property
    def out_dim_factor(self):
        return 1   

Now, we are ready to test the `SimpleMeanLayer` on some DGL graphs. Note that in this example, we **ignore** the edge features since they are not supported.

In [4]:
graph = deepcopy(bg)
h_in = graph.ndata["h"]
layer = SimpleMeanLayer(
            in_dim=in_dim, out_dim=out_dim, 
            activation="relu", dropout=.3, normalization="batch_norm").to(float)
h_out = layer(graph, h_in)

print(h_in.shape)
print(h_out.shape)

torch.Size([7, 5])
torch.Size([7, 11])


## Creating a complex layer with edges

Here, we will show how to create a GNN layer that does a mean aggregation on the neighbouring features, concatenated to the edge features with their neighbours. In that case, only the node features will change, and the network will not update the edge features.

The virtual methods will have different outputs

- `layer_supports_edges`: We want to return `True` since our layer does support edges
- `layer_inputs_edges`: We want to return `True` since our layer does input edges
- `layer_outputs_edges`: We want to return `False` since our layer will not output new edges
- `layer_outdim_factor`: We want to return `1` since the output dimension does not depend on internal parameters.

The example is given below

In [5]:
class ComplexMeanLayer(BaseDGLLayer):
    def __init__(self, in_dim, out_dim, in_dim_edges, activation, dropout, normalization):
        # Initialize the parent class
        super().__init__(   in_dim=in_dim, out_dim=out_dim, activation=activation,
                            dropout=dropout, normalization=normalization)

        # Create the layer with learned parameters. Note the addition
        self.layer = FCLayer(in_dim=in_dim + in_dim_edges, out_dim=out_dim)

    def cat_nodes_edges(self, edges):
        # Create a message "m" by concatenating "h" and "e" for each pair of nodes
        nodes_edges = torch.cat([edges.src["h"], edges.data["e"]], dim=-1)
        return {"m": nodes_edges}

    def get_edges_messages(self, edges): # Simply return the messages on the edges
        return {"m": edges.data["m"]}

    def forward(self, g, h, e):

        # We first concatenate both the node and edge features on the edges
        g.ndata["h"] = h
        g.edata["e"] = e
        g.apply_edges(self.cat_nodes_edges)

        # Then we apply the mean aggregation to generate a message "m"
        g.update_all(message_func=self.get_edges_messages, 
                    reduce_func=dgl.function.mean("m", "h"))

        # Finally we apply the FCLayer, and the non-linearities
        h = g.ndata["h"]
        h = self.layer(h)
        h = self.apply_norm_activation_dropout(h)
        return h

    # Finally, we define all the virtual properties according to how
    # the class works
    @classproperty
    def layer_supports_edges(cls):
        return True

    @property
    def layer_inputs_edges(self):
        return True

    @property
    def layer_outputs_edges(self):
        return False

    @property
    def out_dim_factor(self):
        return 1   

Now, we are ready to test the `ComplexMeanLayer` on some DGL graphs. Note that in this example, we **use** the edge features since they are mandatory.

In [6]:
graph = deepcopy(bg)
h_in = graph.ndata["h"]
e_in = graph.edata["e"]
layer = ComplexMeanLayer(
            in_dim=in_dim, out_dim=out_dim, in_dim_edges=in_dim_edges,
            activation="relu", dropout=.3, normalization="batch_norm").to(float)
h_out = layer(graph, h_in, e_in)

print(h_in.shape)
print(h_out.shape)

torch.Size([7, 5])
torch.Size([7, 11])
